In [1]:
!pip install tqdm

In [6]:
import tensorflow as tf
from ray.rllib.offline.json_reader import JsonReader
import numpy as np
import numpy_indexed as npi
import pandas as pd
#from true_state_viewer import TrueStateTreeGraphViz, display_tree_pairs
from tqdm import trange
import os

In [8]:
path = 'logs/APPO/no_decoy_800000'
input_reader = JsonReader(path)
num_episodes = int(400*10)
num_data_points = num_episodes * 99
state_length = 91
num_actions = 41

states = np.ones((num_data_points, state_length))
states_t = np.ones((num_data_points, state_length+1))
actions_onehot = np.zeros((num_data_points, num_actions))
rewards = np.ones(num_data_points) 
next_states = np.ones((num_data_points, state_length))
next_states_t = np.ones((num_data_points, state_length+1))
inital_states = []

index = 0
for e in trange(num_episodes):
    data = input_reader.next()
    states[index:index+99,:] = data['obs'][0:99]
    states_t[index:index+99,:] = np.concatenate([data['obs'][0:99],np.arange(1,100).reshape(99,1)/100], axis=1)
    next_states[index:index+99,:] = data['obs'][1:100]
    next_states_t[index:index+99,:] = np.concatenate([data['obs'][1:100],np.arange(1,100).reshape(99,1)/100], axis=1)
    rewards[index:index+99] = data['rewards'][0:99]
    actions_onehot[np.arange(index,index+99),data['actions'][0:99]] = 1  
    index += 99
    
    if not data['obs'][0].tobytes() in inital_states:
        inital_states.append(data['obs'][0].tobytes())

2023-03-21 15:40:38,719	WARNING json_reader.py:239 -- Treating input directory as glob patterns: ['/home/adamprice/u75a-Data-Efficient-Decisions/CybORG/CybORG/Notebooks/logs/APPO/no_decoy_800000/*.json', '/home/adamprice/u75a-Data-Efficient-Decisions/CybORG/CybORG/Notebooks/logs/APPO/no_decoy_800000/*.zip']
100%|██████████| 4000/4000 [07:06<00:00,  9.37it/s]


In [10]:
#os.mkdir(path + '/data')
np.save(path + '/data/states.npy', states)
np.save(path + '/data/states_t.npy', states_t)
np.save(path + '/data/next_states.npy', next_states)
np.save(path + '/data/next_states_t.npy', next_states_t)
np.save(path + '/data/afterstates.npy', afterstates)
np.save(path + '/data/rewards.npy', rewards)
np.save(path + '/data/actions_onehot.npy', actions_onehot)

In [ ]:
episode_len = 99

states_seq = np.zeros((num_data_points, episode_len, state_length))
actions_onehot_seq = np.zeros((num_data_points,episode_len, num_actions))

index = 0
for e in trange(num_episodes):
    data = input_reader.next()
    for t in range(1,episode_len):
        states_seq[index+t,0:t,:] = data['obs'][0:t]
        actions_onehot_seq[index+t,0:t,data['actions'][0:t]] = 1  
    index += 99

In [61]:
np.save(path + '/data/states_seq.npy', states_seq)
np.save(path + '/data/actions_onehot_seq.npy', actions_onehot_seq)

In [6]:
episode_len = 99
window = 10

states_seq = np.zeros((num_data_points, window, state_length))
actions_onehot_seq = np.zeros((num_data_points,window, num_actions))

index = 0
for e in trange(num_episodes):
    data = input_reader.next()
    for t in range(1,episode_len-window):
        states_seq[index+t,:,:] = data['obs'][t:t+window]
        actions_onehot_seq[index+t,:,data['actions'][t:t+window]] = 1  
    index += 99

100%|██████████| 2000/2000 [49:44<00:00,  1.49s/it]


In [7]:
np.save(path + '/data/states_seq_10.npy', states_seq)
np.save(path + '/data/actions_onehot_seq_10.npy', actions_onehot_seq)